In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Current Research/Water/water-raw/final_county_water1970-1988.csv'
df_water = pd.read_csv(file_path)

print(df_water.shape)

df_water.head()

(10031, 9)


,countyname,year,month,day,nitrate_mean,nitrate_median,Latitude,Longitude,date_taken
0,Adair,1970,9,10,0.113000,0.1130,41.503820,-94.316260,9/10/1970
1,Adair,1971,4,30,1.170125,1.2095,41.291950,-94.540005,4/30/1971
2,Adair,1971,8,2,0.110000,0.1100,41.301839,-94.483226,8/2/1971
3,Adair,1971,12,13,0.440500,0.4405,41.301839,-94.483226,12/13/1971
4,Adair,1972,2,1,0.956333,0.7000,41.503820,-94.316260,2/1/1972


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Current Research/Water/birth-raw/final_natalityus1970-1988.csv'
df_birth = pd.read_csv(file_path)
df_birth['date_taken'] = pd.to_datetime(df_birth['start'])

import numpy as np

# Add a numeric variable 'x' and set it to NaN for every observation in df1
df_birth['x'] = np.nan
df_birth['d'] = np.nan

print(df_birth.shape)

df_birth.head()

(725756, 103)


,datayear,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,divres,...,countyname,fips,county,pop_2000,Latitude,Longitude,start,date_taken,x,d
0,0,2,3,16,16084,999,9,0,2,4,...,Sioux,19167,Sioux,31589,43.078397,-96.165810,1970-06-25,1970-06-25,NaN,NaN
1,0,2,3,16,16086,999,9,0,2,4,...,Tama,19171,Tama,18103,42.066911,-92.541437,1969-11-10,1969-11-10,NaN,NaN
2,0,2,3,16,16064,999,9,0,2,4,...,Marshall,19127,Marshall,39311,42.031702,-92.937291,1969-10-26,1969-10-26,NaN,NaN
3,0,2,3,16,16016,999,9,0,2,4,...,Cedar,19031,Cedar,18187,41.760598,-91.136340,1970-01-18,1970-01-18,NaN,NaN
4,0,2,3,16,16048,999,9,0,2,4,...,Iowa,19095,Iowa,15671,41.703898,-92.046364,1970-05-07,1970-05-07,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# Function to process each chunk
def process_chunk(chunk, county_groups):
    # Convert date_taken to datetime and int64 for chunk
    chunk['date_taken'] = pd.to_datetime(chunk['date_taken'])

    def find_closest_date(row):
        county = row['countyname']
        date_numeric = row['date_taken'].value  # Convert Timestamp to int64 (nanoseconds)

        # If the county exists in df2, proceed with nearest neighbor search
        if county in county_groups.groups:
            df2_group = county_groups.get_group(county)
            # Create a BallTree for efficient nearest neighbor search based on date_taken
            tree = BallTree(df2_group[['date_taken_numeric']].values, metric='euclidean')
            dist, ind = tree.query([[date_numeric]], k=1)

            # Retrieve the closest date and its corresponding nitrate_median value
            closest_row = df2_group.iloc[ind[0][0]]
            return closest_row['nitrate_median'], (row['date_taken'] - closest_row['date_taken']).days
        return np.nan, np.nan

    # Apply the function to the chunk
    chunk[['nitrate_median', 'd']] = chunk.apply(find_closest_date, axis=1, result_type='expand')
    return chunk

# Prepare df2 (df_water) for searching (convert to datetime and int64)
df_water['date_taken'] = pd.to_datetime(df_water['date_taken'])
df_water['date_taken_numeric'] = df_water['date_taken'].values.astype('int64')

# Group df_water by countyname for efficient lookup
county_groups = df_water.groupby('countyname')

# Estimate optimal chunk size based on available memory
memory_usage_per_row = df_birth.memory_usage(deep=True).sum() / len(df_birth)
available_memory = 2 * 1024**3  # Assume 2 GB of available memory for processing
optimal_chunk_size = int(available_memory / memory_usage_per_row)

print(f"Estimated optimal chunk size: {optimal_chunk_size}")

# Start processing df_birth in chunks
n_chunks = (len(df_birth) // optimal_chunk_size) + 1  # Number of chunks to process
for i, start in enumerate(range(0, len(df_birth), optimal_chunk_size)):
    # Get the chunk of df_birth to process
    chunk = df_birth.iloc[start:start + optimal_chunk_size].copy()  # Copy to avoid modifying the original df_birth

    # Process the chunk
    processed_chunk = process_chunk(chunk, county_groups)

    # Write to CSV incrementally
    if i == 0:
        processed_chunk.to_csv('linked_birth_water_county1970-1988.csv', mode='w', index=False)  # First chunk writes header
    else:
        processed_chunk.to_csv('linked_birth_water_county1970.csv', mode='a', header=False, index=False)  # Append subsequent chunks


Estimated optimal chunk size: 1862068


In [ ]:
import pandas as pd

file_path = '/content/linked_birth_water_county1970-1988.csv'
df_linked = pd.read_csv(file_path)

print(df_linked.shape)

df_linked.head()

(725756, 104)


,datayear,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,divres,...,fips,county,pop_2000,Latitude,Longitude,start,date_taken,x,d,nitrate_median
0,0,2,3,16,16084,999,9,0,2,4,...,19167,Sioux,31589,43.078397,-96.165810,1970-06-25,1970-06-25,NaN,-89.0,1.1125
1,0,2,3,16,16086,999,9,0,2,4,...,19171,Tama,18103,42.066911,-92.541437,1969-11-10,1969-11-10,NaN,-57.0,0.1130
2,0,2,3,16,16064,999,9,0,2,4,...,19127,Marshall,39311,42.031702,-92.937291,1969-10-26,1969-10-26,NaN,-98.0,0.0000
3,0,2,3,16,16016,999,9,0,2,4,...,19031,Cedar,18187,41.760598,-91.136340,1970-01-18,1970-01-18,NaN,-577.0,0.0000
4,0,2,3,16,16048,999,9,0,2,4,...,19095,Iowa,15671,41.703898,-92.046364,1970-05-07,1970-05-07,NaN,-179.0,0.0250
